# EM LOOP

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

uf = 'AC - Acre'
var_1 = 'Bloco 2 - Existência de banheiro'
var_2 = 'Bloco 1 - Estado cadastral da família'

# Configurar o WebDriver usando o ChromeDriverManager para gerenciar o ChromeDriver
service = ChromeService(executable_path=ChromeDriverManager().install())

# Inicializar o WebDriver
driver = webdriver.Chrome(service=service)

# Abrir a página
driver.get('https://cecad.cidadania.gov.br/tab_cad.php')

# Carregar e criar a lista com os municípios
municipios_ = pd.read_excel('municipios.xlsx')
municipios = municipios_[municipios_['ds_uf'] == uf]
# Passando para lista
municipios = municipios['municipio'].tolist()

# Esperar até que o botão de filtros esteja clicável
filtro_label = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'label.onoffswitch-label[for="myonoffswitch"]'))
)

# Clicar no botão de filtros
filtro_label.click()

# Esperar até que os filtros estejam visíveis
WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.ID, 'filtros'))
)

# Selecionar os checkboxes de faixa de renda
checkbox_values_renda = ["1", "2", "3"]
for value in checkbox_values_renda:
    checkbox = driver.find_element(By.CSS_SELECTOR, f'input[name="filtros[fx_rfpc][]"][value="{value}"]')
    if not checkbox.is_selected():
        checkbox.click()

# Selecionar os checkboxes de parentesco
checkbox_values_parentesco = ["11", "12"]
for value in checkbox_values_parentesco:
    checkbox = driver.find_element(By.CSS_SELECTOR, f'input[name="filtros[fx_idade][]"][value="{value}"]')
    if not checkbox.is_selected():
        checkbox.click()

# Esperar até que o seletor de estados esteja presente
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, 'estadoSAGIUFMU'))
)

uf = 'AC - Acre'
# Selecionar um estado
estado_select = Select(driver.find_element(By.ID, 'estadoSAGIUFMU'))
estado_select.select_by_visible_text(uf)

# Inicializar DataFrames vazios
df_familia = pd.DataFrame()
df_pessoa = pd.DataFrame()

# Função para processar uma tabela
def process_table(table, uf, munic):
    rows = table.find_elements(By.TAG_NAME, 'tr')
    data = []
    headers = None
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, 'td')
        if not cells:
            header_cells = row.find_elements(By.TAG_NAME, 'th')
            if headers is None:
                headers = [cell.text for cell in header_cells if "colspan" not in cell.get_attribute('outerHTML')]
            else:
                headers.extend([cell.text for cell in header_cells if "colspan" not in cell.get_attribute('outerHTML')])
        else:
            first_cell = row.find_element(By.TAG_NAME, 'th').text
            data.append([uf, munic, first_cell] + [cell.text for cell in cells])
    
    if headers:
        headers = [header for header in headers if header]
        headers = ["UF", "Município"] + headers
        if "TOTAL" in headers:
            headers.remove("TOTAL")
            headers.append("TOTAL")
        return pd.DataFrame(data, columns=headers)
    else:
        raise ValueError("Headers não encontrados na tabela.")

# Loop sobre os municípios
for munic in municipios:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'municipioSAGIUFMU'))
    )
    
    municipio_select = Select(driver.find_element(By.ID, 'municipioSAGIUFMU'))
    municipio_select.select_by_visible_text(munic)
    
    coluna_select = Select(driver.find_element(By.NAME, 'var1'))
    coluna_select.select_by_visible_text(var_1)
    
    linha_select = Select(driver.find_element(By.NAME, 'var2'))
    linha_select.select_by_visible_text(var_2)
    
    valor_absoluto_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), '% Total')]"))
    )
    
    valor_absoluto_button.click()
    
    tabela_carregada = WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, '#content .table'))
    )
    
    tables = driver.find_elements(By.CSS_SELECTOR, '#content .table')
    
    if len(tables) >= 2:
        table_familia = tables[0]
        table_pesso = tables[1]
        
        df_temp_familia = process_table(table_familia, uf, munic)
        df_familia = pd.concat([df_familia, df_temp_familia], ignore_index=True)
        
        df_temp_pesso = process_table(table_pesso, uf, munic)
        df_pessoa = pd.concat([df_pessoa, df_temp_pesso], ignore_index=True)
    else:
        raise ValueError("Não foram encontradas duas tabelas na página.")

# Exibir os dados
display(df_familia)
display(df_pessoa)

# Fechar o navegador
driver.quit()


,UF,Município,Estado cadastral da família,Sim,Não,Sem Resposta,TOTAL
0,AC - Acre,Acrelândia,Sem Registro Civil,"0,00%","0,00%","0,00%","0,00%"
1,AC - Acre,Acrelândia,Cadastrado,"84,00%","13,78%","2,22%","100,00%"
2,AC - Acre,Acrelândia,Sem Resposta,"0,00%","0,00%","0,00%","0,00%"
3,AC - Acre,Acrelândia,TOTAL,"84,00%","13,78%","2,22%","100,00%"
4,AC - Acre,Assis Brasil,Sem Registro Civil,"0,00%","0,00%","0,00%","0,00%"
...,...,...,...,...,...,...,...
83,AC - Acre,Tarauacá,TOTAL,"54,89%","44,74%","0,38%","100,00%"
84,AC - Acre,Xapuri,Sem Registro Civil,"0,00%","0,00%","0,00%","0,00%"
85,AC - Acre,Xapuri,Cadastrado,"75,26%","24,23%","0,52%","100,00%"
86,AC - Acre,Xapuri,Sem Resposta,"0,00%","0,00%","0,00%","0,00%"


,UF,Município,Estado cadastral da família,Sim,Não,Sem Resposta,TOTAL
0,AC - Acre,Acrelândia,Sem Registro Civil,"0,00%","0,00%","0,00%","0,00%"
1,AC - Acre,Acrelândia,Cadastrado,"85,32%","12,70%","1,98%","100,00%"
2,AC - Acre,Acrelândia,Sem Resposta,"0,00%","0,00%","0,00%","0,00%"
3,AC - Acre,Acrelândia,TOTAL,"85,32%","12,70%","1,98%","100,00%"
4,AC - Acre,Assis Brasil,Sem Registro Civil,"0,00%","0,00%","0,00%","0,00%"
...,...,...,...,...,...,...,...
83,AC - Acre,Tarauacá,TOTAL,"54,50%","45,16%","0,35%","100,00%"
84,AC - Acre,Xapuri,Sem Registro Civil,"0,00%","0,00%","0,00%","0,00%"
85,AC - Acre,Xapuri,Cadastrado,"76,78%","22,27%","0,95%","100,00%"
86,AC - Acre,Xapuri,Sem Resposta,"0,00%","0,00%","0,00%","0,00%"


In [7]:
def convert_columns_to_float(df):
    df = df.replace('%', '', regex=True)
    df = df.replace(',', '.', regex=True)
    for column in df.columns:
        try:
            df[column] = df[column].astype(float)
        except ValueError:
            pass  # Se a conversão falhar, a coluna permanece como está
    return df


df_familia = convert_columns_to_float(df_familia)
df_pessoa = convert_columns_to_float(df_pessoa)

print("Família:")
display(df_familia)
print("="*60)
print("Pessoa:")
display(df_pessoa)


Família:


,UF,Município,Estado cadastral da família,Sim,Não,Sem Resposta,TOTAL
0,AC - Acre,Acrelândia,Sem Registro Civil,0.00,0.00,0.00,0.0
1,AC - Acre,Acrelândia,Cadastrado,84.00,13.78,2.22,100.0
2,AC - Acre,Acrelândia,Sem Resposta,0.00,0.00,0.00,0.0
3,AC - Acre,Acrelândia,TOTAL,84.00,13.78,2.22,100.0
4,AC - Acre,Assis Brasil,Sem Registro Civil,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...
83,AC - Acre,Tarauacá,TOTAL,54.89,44.74,0.38,100.0
84,AC - Acre,Xapuri,Sem Registro Civil,0.00,0.00,0.00,0.0
85,AC - Acre,Xapuri,Cadastrado,75.26,24.23,0.52,100.0
86,AC - Acre,Xapuri,Sem Resposta,0.00,0.00,0.00,0.0


Pessoa:


,UF,Município,Estado cadastral da família,Sim,Não,Sem Resposta,TOTAL
0,AC - Acre,Acrelândia,Sem Registro Civil,0.00,0.00,0.00,0.0
1,AC - Acre,Acrelândia,Cadastrado,85.32,12.70,1.98,100.0
2,AC - Acre,Acrelândia,Sem Resposta,0.00,0.00,0.00,0.0
3,AC - Acre,Acrelândia,TOTAL,85.32,12.70,1.98,100.0
4,AC - Acre,Assis Brasil,Sem Registro Civil,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...
83,AC - Acre,Tarauacá,TOTAL,54.50,45.16,0.35,100.0
84,AC - Acre,Xapuri,Sem Registro Civil,0.00,0.00,0.00,0.0
85,AC - Acre,Xapuri,Cadastrado,76.78,22.27,0.95,100.0
86,AC - Acre,Xapuri,Sem Resposta,0.00,0.00,0.00,0.0


In [10]:
# Salvando em um arquivo Excel com duas abas
with pd.ExcelWriter('dados/dados.xlsx') as writer:
    df_familia.to_excel(writer, sheet_name='familia', index=False)
    df_pessoa.to_excel(writer, sheet_name='pessoa', index=False)
